In [1]:
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
with open('packagefordeployment/tokenizer0421.json') as f:
    data = json.load(f)
    ntokenizer = tf.keras.preprocessing.text.tokenizer_from_json(data)

In [6]:
#pretrainedmodel = tf.keras.models.load_model('sentimentdetectmodel.h5')
pretrainedmodel = tf.keras.models.load_model('pretrainedmodel/sentimentdetectmodel0421.h5')


In [7]:
testsentences = ['it was very good', 'i love my family', 'it was not very good'
                 ,'Go with something that is better quality than this one'
                 ,'i do not love ui work', ' i adore you', 'i luv you'
                ]
testsequences = ntokenizer.texts_to_sequences(testsentences)
testpadded = pad_sequences(testsequences, maxlen=40)
answer = pretrainedmodel.predict(testpadded)
answer

array([[0.9891399 ],
       [0.9305    ],
       [0.00625599],
       [0.01510814],
       [0.02991026],
       [0.83468324],
       [0.882621  ]], dtype=float32)

In [8]:
import numpy as np
def run(data):
    try:
        data = json.loads(data)
        sentences = data['sentences']

        testsequences = ntokenizer.texts_to_sequences(sentences)
        testpadded = pad_sequences(testsequences, maxlen=40)
        answer = pretrainedmodel.predict(testpadded)
        answer = answer.reshape(len(sentences))
        answer = np.array(answer, dtype=np.str)
        #print(answer)
        #res = dict(zip(sentences, answer)) 
        #print(res)
        answer = list(answer)
        
        #result = {"sentiment":str(answer[0])}
        return json.dumps(answer)
        
    except Exception as e:
        error = str(e)
        return error

In [9]:
mydict = {}
mydict['sentences'] = testsentences
run(json.dumps(mydict))

'["0.9891399", "0.9305", "0.00625599", "0.015108142", "0.02991026", "0.83468324", "0.882621"]'

In [89]:
import pandas as pd
def run(inputdata):
    try:
        data = json.loads(inputdata)
        sentences = data['sentences']
        inputdf = pd.DataFrame(sentences,columns=['inputdata'])
        #print(inputdf)
        testsequences = ntokenizer.texts_to_sequences(sentences)
        testpadded = pad_sequences(testsequences, maxlen=40)
        answer = pretrainedmodel.predict(testpadded)
        answerdf = pd.DataFrame(answer, columns=['result'])
       
        #print(answerdf)
        finalans = inputdf.join(answerdf)
        finalans.index = finalans['inputdata']
        finalans = finalans.drop(columns=['inputdata'])
        #print(finalans)
        #result = {"sentiment":str(answer[0])}
        return json.dumps(finalans.to_dict())
        
        #return data +
        # You can return any data type, as long as it is JSON serializable.

    except Exception as e:
        error = str(e)
        return error

In [92]:
myobj = ['this is good', 'this is very good', 'i am very happy','Go with something that is better quality than this one']
mydict = {}
mydict['sentences'] = myobj
print(json.dumps(mydict))
result = run(json.dumps(mydict))
print(result)


{"sentences": ["this is good", "this is very good", "i am very happy", "Go with something that is better quality than this one"]}
{"result": {"this is good": 0.9836187958717346, "this is very good": 0.994709849357605, "i am very happy": 0.9907805323600769, "Go with something that is better quality than this one": 0.015108142048120499}}


In [91]:
print(tf.__version__)
print(pd.__version__)

2.1.0
1.0.3


In [88]:
pretrainedmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 100)           108335700 
_________________________________________________________________
bidirectional (Bidirectional (None, 40, 128)           84480     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 108,465,621
Trainable params: 129,921
Non-trainable params: 108,335,700
_________________________________________________________________
